## Eigenfaces

Eigenfaces (en español caras propias) es el nombre dado a un conjunto de autovectores cuando se utiliza en el problema de visión artificial del reconocimiento de rostros humanos. Matthew Turk y Alex Pentland lo propusieron en su [paper en la clasificación de caras](https://doi.org/10.1162/jocn.1991.3.1.71).

Aquí se muestra como hacer el cálculo del Análisis en Componentes Principales para los datos de los rostros, y luego aplicarlos para la reducción del espacio. Con este nuevo espacio se realizan dos tareas: reconstrucción y clasificación.

In [ ]:
import numpy as np
import scipy.io as sio
from matplotlib import pyplot as plt
from sklearn.model_selection import StratifiedKFold

Definición de funciones 

In [ ]:
def svd_reducida(A,k='max',tol=1e-12):
    # A completar por ustedes!

def splitDataset(data, label):
    # split dataset en entrenamiento y test
    skf = StratifiedKFold(n_splits=3, shuffle=True, random_state=1)
    for train, test in skf.split(data.T, label.T):
        break
    train_data = data[:,train]
    train_lab = label[:,train]
    test_data = data[:,test]
    test_lab = label[:,test]
    
    return train_data, train_lab, test_data, test_lab

def showPic(data, idx, dx=38):
    # graficamos una rotro
    v = data[:,idx] # primera columna
    m = v.reshape((dx,dx)).T
    plt.imshow(m, cmap=plt.cm.gray)


La lectura del archivo de datos nos devuelve dos matrices, una correspondiente a las imágenes de los rostros y la otra a un label que indica a cual
persona pertenece la imágen.

<img src="image_vector.png" />

Como vemos en la figura, las imágenes se convirtieron a vector, con $N=38$ para este set de datos y por lo tanto tenemos $n = N^2 =1444$ variables.

In [ ]:
mat = sio.loadmat('base_40_38_10.mat')
# data es una matriz de 1444 x 380.
# Corresponde a figuras de caras de tamanio 38x38 pixeles
data = mat['data']
label = mat['label']
# extraemos la dimensionalidad de data, donde n es la cantidad de ejemplos, y d la dimensión del espacio.
d, n = data.shape

showPic(data, 51)

print(label.max())

## Análisis de Componentes Principales (ACP o PCA en inglés)

Vamos a hacer una modificación respecto del formalismo que usamos en la sección precedente, ya que vamos a acomodar las muestras (los rostros) como columnas.
Tampoco vamos a dividir por el desvío estándar ya que las imágenes están acotadas en sus valores de niveles de gris de los pixeles (y por lo tanto están en la misma escala).


Primero centramos los valores de las imágenes, restando la media.
Luego calculamos la matriz de covarianzas:


$C = \frac{1}{n} \sum_{i=1}^n (x_i - \mu) (x_i - \mu)^{T}$

Finalmente encontramos los autovalores y autovectores de esta matriz, los cuales ordenamos de mayor a menor valor.

In [ ]:
def calculoACP(data):
    dx = 38
    d, n = data.shape
    m=np.mean(data, axis=1)
    print(m)
    plt.imshow(m.reshape((dx,dx)).T, cmap=plt.cm.gray)
    
    X = data - np.tile(m.reshape((len(m), 1)), (1, n))
    Mcov = np.dot(X,X.T) / n # Covariance Matrix

    U, D, V = svd_reducida(A)# Completen con su implementación de SVD
    
    # ordenamos los autovalores de mayor a menor
    idx = np.argsort (- D )
    D = D[idx]
    U = U[:, idx]

    return D, U, X, m

In [ ]:
D, U, data_ref, m = calculoACP(data)

Veamos como se ven las imágenes que quedan representadas en las columnas de U

In [ ]:
showPic(U, 0)
print(U[:,0])

El espacio de proyección del ACP esta compuesto por el vector V que es de tamaño $n \times n$.

El próximo paso busca la reducción del espacio de proyección, para quedarnos con aquellos autovectores en V que acumulen la mayor cantidad de información posible en las distintas direcciones.

Para ello se hace un cómputo de la varianza acumulada en el vector D, y se selecciona una cantidad que signifique representar un 95 % de la información.

In [ ]:
ratio = np.cumsum(D) / np.sum(D)
plt.plot(ratio)
x = np.where(ratio > 0.95)[0]
M = x[0]

print('Cantidad de autovectores de representación al 95 %: ', M)

### Reconstrucción de un rostro

El hecho que quedarse con menos autovectores para la proyección del espacio, conlleva a una reducción de almacenamiento de la información, pero al mismo tiempo a cometer un error al tratar de reconstruir la imagen original.

En este tramo de código representamos visualmente la imagen original y la reconstruida con M autovectores.

In [ ]:
# Reconstruccion
idx_im = 5
im_orig = data_ref[:,idx_im]
cpM = U[:,0:M].T @ im_orig 

print(cpM.shape)

im_rec = U[:,0:M] @ cpM 

fig, axes = plt.subplots(1,2)
axes[0].imshow(im_orig.reshape((38,38)).T, cmap=plt.cm.gray)
axes[1].imshow(im_rec.reshape((38,38)).T + m.reshape((38,38)).T, cmap=plt.cm.gray)
plt.show()

## Clasificación de nuevas imágenes

La tarea de clasificación en predecir a quien de las personas de la base de conocimientos pertenece un rostro de testing. Esto lo vamos a realizar gracias a proyectar el rostro de entrada al espacio de ACP y calcular por distancias, cual es rostro más cercano.

En primer lugar separamos 

In [ ]:
# clasificacion
train_data, train_lab, test_data, test_lab = splitDataset(data, label)
Dt, Ut, train_ref, m = calculoACP(train_data)

Luego elegimos la cantidad de componentes que vamos a usar

In [ ]:
ratio = np.cumsum(Dt) / np.sum(Dt)
plt.plot(ratio)
x = np.where(ratio > 0.95)[0]
M = x[0]
print('Cantidad de autovectores de representación al 95 % de la base de entrenamiento: ', M)

Y por último elegimos la etiqueta en base a la proyección de la imagen

In [ ]:
data_clf = train_ref.T @ Ut[:,0:M]   # proyectamos a la base de entrenamiento, de los cuales conocemos a que persona pertenece

input_test = test_data[:,0] # vamos a clasificar el primer sujeto de la base de test
test_acp = (input_test - m) @ Ut[:,0:M]    # le resto la media y proyecto en el espacio reducido de Vt
Q = np.tile(test_acp.reshape((1,-1)), (data_clf.shape[0], 1))  
dist = np.linalg.norm(data_clf - Q, axis=1)    # calculo las distancias a cada una de las imágenes de conocimientos proyectadas en el espacio ACP.
y = np.argmin(dist)                             # clasificar por el más cercano

if test_lab[0][0] == train_lab[0][y]:
    print('Clasificacion correcta')
else:
    print('clasificacion incorrecta')

### Ejercicio

Ejecutar el script anterior pero evaluando el error de clasificaciones correctas e incorrectas para todo el test_data.
